In [1]:
import pickle
import polars as pl
from datetime import datetime, timedelta
import numpy as np
from glob import glob
from tqdm import tqdm
import concurrent.futures
import os
import sys
import pandas as pd
import geopandas as gpd

from tqdm import tqdm

In [2]:
idprov='95'
df_distance=pd.read_csv('/data/raw/processed/idsub_distance_selindo.csv')
data_cluster=gpd.read_file(f'/data/ksa/00_Data_Input/cluster_data/cluster_{idprov}_ok.gpkg')

In [3]:
prov_pickle_wide=glob(f'/data/ksa/04_Data_Preprocessing/{idprov}/01_imputation/wide_data/sampling_VH_*.pkl')

In [4]:
dfs = []

for pkl_file in tqdm(prov_pickle_wide):
    df = pd.read_pickle(pkl_file).dropna()
    df['idkabkota']=df.idsubsegment.apply(lambda y: y[:4])
    df['observation']=df.observation.apply(lambda y: int(y))
    pl_df=pl.from_pandas(df
                        )
    #print(pl_df.columns)
    #pl_df = pl_df.with_columns([pl.col("observation ").cast(pl.Int64)])
    dfs.append(pl_df)
pl_df_combined = pl.concat(dfs, how="vertical")
#pl_df_combined = pl.from_pandas(combined_df)


100%|██████████| 4/4 [00:00<00:00,  9.64it/s]


In [5]:
#just for 94
data_cluster['cluster_data']=0
cluster_data=pl.from_pandas(data_cluster[['idkab','cluster_data']])
pl_df_combined=pl_df_combined.join(cluster_data,left_on='idkabkota',right_on='idkab',how='left')

In [10]:
pl_df_combined

idpoint,idsubsegment,idsegment,nth,periode,observation,class,MGRS,VH_30,VH_29,VH_28,VH_27,VH_26,VH_25,VH_24,VH_23,VH_22,VH_21,VH_20,VH_19,VH_18,VH_17,VH_16,VH_15,VH_14,VH_13,VH_12,VH_11,VH_10,VH_9,VH_8,VH_7,VH_6,VH_5,VH_4,VH_3,VH_2,VH_1,VH_0,idkabkota,cluster_data
str,str,str,i64,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64
"""950104309A1#01""","""950104309A1""","""950104309""",1,"""2022_03""",6,"""NP""","""54LVS""",-15.267017,-14.505255,-15.425903,-16.468752,-15.627224,-15.67943,-15.264399,-16.188999,-15.769554,-16.772329,-17.113848,-16.765179,-19.450386,-17.435328,-16.587332,-16.501287,-15.820472,-15.738201,-15.050714,-14.105463,-14.744896,-14.693724,-15.387698,-15.514237,-15.765416,-16.104027,-17.272125,-16.202618,-14.917972,-16.494663,-16.100204,"""9501""",0
"""950104309A1#02""","""950104309A1""","""950104309""",1,"""2022_03""",6,"""NP""","""54LVS""",-15.545425,-15.136057,-15.97317,-17.092134,-16.018103,-16.06563,-15.707749,-15.444011,-16.835285,-16.94595,-17.10266,-16.431044,-19.080738,-17.223346,-16.667591,-16.310491,-15.295516,-15.883364,-15.051763,-13.971258,-14.907809,-15.666601,-15.087698,-15.614631,-15.689068,-16.272843,-17.239288,-16.640381,-16.259346,-16.897736,-16.604695,"""9501""",0
"""950104309A1#03""","""950104309A1""","""950104309""",1,"""2022_03""",6,"""NP""","""54LVS""",-15.081805,-15.582939,-15.974492,-16.269899,-16.528959,-16.576262,-16.493195,-16.496466,-17.144741,-17.412956,-17.282543,-16.714308,-18.088755,-16.847383,-16.576366,-15.831636,-15.072727,-14.711138,-14.698113,-14.102802,-14.912554,-15.311652,-16.18298,-15.937101,-15.723689,-16.45885,-17.293987,-16.61751,-16.209307,-16.555347,-16.168014,"""9501""",0
"""950104309A1#04""","""950104309A1""","""950104309""",1,"""2022_03""",6,"""NP""","""54LVS""",-15.190509,-15.490117,-15.608857,-16.205917,-15.366982,-16.260111,-16.710827,-16.205597,-17.819567,-18.102028,-17.404493,-15.598152,-18.840685,-16.956352,-16.709345,-16.044848,-15.358318,-14.510735,-15.340473,-16.003563,-15.585822,-15.052164,-16.638023,-16.042763,-16.385624,-15.943397,-15.712965,-15.754011,-15.129875,-15.824692,-16.214305,"""9501""",0
"""950104309A1#05""","""950104309A1""","""950104309""",1,"""2022_03""",6,"""NP""","""54LVS""",-15.919489,-16.890448,-16.223655,-16.522982,-15.240166,-16.376899,-17.043068,-15.80102,-18.248425,-17.090389,-17.169501,-15.574532,-18.647285,-16.956173,-17.257921,-15.909712,-15.124194,-13.601419,-14.709009,-15.096437,-14.959707,-15.142719,-15.71074,-15.664811,-16.193209,-15.981299,-15.515672,-16.416261,-16.837654,-16.892063,-17.251857,"""9501""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""950201003C3#21""","""950201003C3""","""950201003""",12,"""2023_30""",6,"""NV""","""54MVT""",-13.868608,-13.662426,-14.59272,-13.557177,-12.090015,-13.528204,-14.6435,-13.936707,-13.007035,-12.299778,-12.341965,-11.913018,-12.414866,-13.297618,-12.344851,-17.368162,-15.596712,-16.105923,-16.044212,-14.943595,-17.336386,-15.635265,-14.562532,-15.053845,-15.372893,-13.903325,-15.008942,-15.75469,-15.480909,-15.697531,-15.941812,"""9502""",0
"""950201003C3#22""","""950201003C3""","""950201003""",12,"""2023_30""",6,"""NV""","""54MVT""",-14.220582,-14.394161,-14.994831,-13.972037,-13.354317,-13.471991,-13.391424,-13.167938,-12.747548,-12.30275,-13.896383,-10.821289,-13.354507,-14.393958,-14.5887,-18.662056,-16.771036,-16.248566,-16.968658,-17.360718,-16.730791,-16.07628,-14.401028,-15.635299,-15.803896,-15.919836,-15.524071,-15.787477,-14.939304,-14.144947,-14.067695,"""9502""",0
"""950201003C3#23""","""950201003C3""","""950201003""",12,"""2023_30""",6,"""NV""","""54MVT""",-14.56641,-15.32053,-14.75309,-14.700775,-15.092854,-14.357133,-13.482676,-14.133934,-13.746225,-14.468336,-13.947365,-11.391563,-13.817242,-15.024438,-15.647329,-18.865162,-17.089854,-17.133507,-16.84449,-16.044239,-16.128651,-16.053528,-15.455608,-15.952008,-16.339773,-15

## Filter Distance

In [12]:
df_distance_group=df_distance.groupby('idsubsegment')[['distance_m']].max().reset_index()\
   .query('distance_m<=20').query('idsubsegment.str.slice(0,2)==@idprov')

KeyError: 'idsubsegment'

In [11]:
df_distance_group

,idsubsegmen_repair,distance_m


In [7]:
ls_idsubsegmen=df_distance_group.idsubsegmen_repair.to_list()

In [8]:
pl_train1=pl_df_combined.filter(pl.col("idsubsegment").is_in(ls_idsubsegmen))

In [9]:
pl_train1.shape

(0, 41)

## Create the variance per observation per VH_0 per cluster

In [22]:
dt_var=pl_train1.group_by(['periode','idsubsegment','cluster_data','observation']).agg([
    pl.col("VH_0").var().alias("Sigma0VH_variance")])

In [23]:
df_sample_train=[]
for cl in cluster_data.to_pandas().cluster_data.unique():
    dt_var_calc=dt_var.filter(pl.col("cluster_data")==cl)
    for obs in dt_var_calc.select(pl.col("observation").unique()).to_pandas().observation:
        print('Cluster:',cl,' observation: ',obs)
        q10_obs=dt_var_calc.filter(pl.col("observation")==obs).select(pl.col("Sigma0VH_variance").quantile(0.30)).item()
        df_true = dt_var_calc.filter(pl.col("observation")==obs).filter(pl.col("Sigma0VH_variance") <= q10_obs)
        df_false=dt_var_calc.filter(pl.col("Sigma0VH_variance") > q10_obs).sample(fraction=0.2,with_replacement=False)
        df_rows=pl.concat([df_true,df_false], how="vertical")
        print(df_rows.shape[0])
        
        df_sample_train.append(df_rows)
df_sample_train=pl.concat(df_sample_train,how='vertical')

Cluster: 0  observation:  0
679
Cluster: 0  observation:  1
529
Cluster: 0  observation:  2
489
Cluster: 0  observation:  3
580
Cluster: 0  observation:  4
570
Cluster: 0  observation:  5
408
Cluster: 0  observation:  6
1362


In [24]:
pl_train_2=pl_train1.join(df_sample_train,on=['periode','idsubsegment','cluster_data','observation'],how='inner')

In [25]:
with open(f"/data/ksa/04_Data_Preprocessing/training-test/{idprov}/training_VH.pkl",'wb') as file:
    pickle.dump(pl_train_2,file)

In [26]:
with open(f"/data/ksa/04_Data_Preprocessing/training-test/{idprov}/fulldataset_VH.pkl",'wb') as file:
    pickle.dump(pl_df_combined,file)